In [ ]:
from pathlib import Path

# Basis-Verzeichnis: dort, wo dein Notebook liegt
BASE_DIR = Path().resolve()  # z.B. .../Erdbeeren_yolo/Riseholme/Riseholme-2021

# Relativer Pfad zu den ripe-Bildern
image_dir = BASE_DIR / 'Data' / 'Normal' / 'Ripe'
# Relativer Pfad, wo die Labels hinkommen sollen
label_dir = BASE_DIR / 'labels' / 'Ripe'

# Existenzauschecken und Ordner anlegen
if not image_dir.is_dir():
    raise FileNotFoundError(f"{image_dir} existiert nicht")
label_dir.mkdir(parents=True, exist_ok=True)

# Klasse 0 = "ripe"
class_id = 0
count = 0

# Jedes Bild → eine YOLO-Label mit Full-Image-Box
for img_path in image_dir.iterdir():
    if img_path.suffix.lower() in {'.jpg', '.jpeg', '.png'}:
        txt_path = label_dir / f"{img_path.stem}.txt"
        with open(txt_path, 'w') as f:
            # x_center=0.5, y_center=0.5, width=1.0, height=1.0
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
        count += 1

print(f"✅ {count} Labels erstellt in {label_dir}")
C:\Users\Maxi\Documents\Forschsem\Riseholme-2021
C:\Users\Maxi\Documents\Forschsem\Erdbeeren\Versuch_2\Erdbeeren_yolo\Riseholme\Erdbeeren_training.ipynb


FileNotFoundError: C:\Users\Maxi\Documents\Forschsem\Erdbeeren\Versuch_2\Erdbeeren_yolo\Riseholme\Data\Normal\Ripe existiert nicht

datensatz splitten

In [ ]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil

BASE_DIR    = Path().resolve()                           # …/Riseholme-2021
IMG_SRC     = BASE_DIR / 'Data' / 'Normal' / 'Ripe'      # deine Original-Bilder
LBL_SRC     = BASE_DIR / 'labels' / 'Ripe'               # deine erzeugten .txt

# Zielordner für YOLO
IMG_TRAIN   = BASE_DIR / 'images' / 'train'
IMG_VAL     = BASE_DIR / 'images' / 'val'
LBL_TRAIN   = BASE_DIR / 'labels' / 'train'
LBL_VAL     = BASE_DIR / 'labels' / 'val'

# Ordner anlegen
for d in (IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL):
    d.mkdir(parents=True, exist_ok=True)

# Dateinamen (Stem) sammeln und splitten
files = [f.stem for f in IMG_SRC.iterdir() if f.suffix.lower() in {'.png','.jpg'}]
train, val = train_test_split(files, test_size=0.2, random_state=42)

# Kopieren
for f in train:
    shutil.copy(IMG_SRC/ f"{f}.png", IMG_TRAIN/ f"{f}.png")
    shutil.copy(LBL_SRC/ f"{f}.txt", LBL_TRAIN/ f"{f}.txt")
for f in val:
    shutil.copy(IMG_SRC/ f"{f}.png", IMG_VAL/ f"{f}.png")
    shutil.copy(LBL_SRC/ f"{f}.txt", LBL_VAL/ f"{f}.txt")

print(f"Train: {len(train)} | Val: {len(val)}")


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')             # oder yolov8s.pt, -m.pt etc.
model.train(
    data='data.yaml',                  # deine Konfig
    epochs=50,                         # je nach Bedarf
    imgsz=640,
    batch=16,
    augment=True,
    project='runs/train',              # Ausgabeordner
    name='ripe_only'                   # Lauf-Name
)
